In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import pydeck as pdk
from collections import Counter

In [2]:
def load_dataset():
    df = pd.read_csv("data.csv")
    df = df[pd.notnull(df['longitude']) & pd.notnull(df['latitude'])]
    return df

def load_geo_subset(df):
    geo_df = df[["longitude", "latitude"]]
    geo_df.dropna(inplace=True)
    return geo_df

def city_list(df):
    return sorted(list(df['city_or_county'].unique()))

def load_city_subset(df, city):
    subdf = df[df['city_or_county'] == city].copy()
    subdf = subdf[["longitude", "latitude", "address", "n_killed", "n_injured"]]
    subdf.dropna(inplace=True)
    return subdf

In [3]:
df = load_dataset()
df.head()

,Unnamed: 0,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url_fields_missing,congressional_district,...,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,state_house_district,state_senate_district
0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,False,14.0,...,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,NaN,NaN
1,1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,False,43.0,...,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,62.0,35.0
2,2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,False,9.0,...,NaN,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,56.0,13.0
3,3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,False,6.0,...,NaN,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,40.0,28.0
4,4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,False,6.0,...,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,62.0,27.0


# Find the column need preprocess

In [4]:
# columns = df.columns
# columns_to_process = df[columns].isnull().any()
# print(columns_to_process)
# df.dtypes.to_dict()

# Preprocess Column by Fill Nan and split to map and save to new column

In [5]:
def get_user_mapping(txt):
    if txt == "NA":
        return {}
    mapping = {}
    for d in txt.split("||"):
        try:
            key = d.split("::")[0]
            val = d.split("::")[1]
            if key not in mapping:
                mapping[key] = val
        except:
            pass
    return mapping

df['participant_type'] = df['participant_type'].fillna("NA")
df['participant_type_map'] = df['participant_type'].apply(lambda x : get_user_mapping(x))
df['participant_age'] = df['participant_age'].fillna("NA")
df['participant_age_map'] = df['participant_age'].apply(lambda x : get_user_mapping(x))
df['participant_age_group'] = df['participant_age_group'].fillna("NA")
df['participant_age_group_map'] = df['participant_age_group'].apply(lambda x : get_user_mapping(x))
df['participant_gender'] = df['participant_gender'].fillna("NA")
df['participant_gender_map'] = df['participant_gender'].apply(lambda x : get_user_mapping(x))


In [6]:
df['participant_age_group_map'].head()

0    {'0': 'Adult 18+', '1': 'Adult 18+', '2': 'Adu...
1    {'0': 'Adult 18+', '1': 'Adult 18+', '2': 'Adu...
2    {'0': 'Adult 18+', '1': 'Adult 18+', '2': 'Adu...
3    {'0': 'Adult 18+', '1': 'Adult 18+', '2': 'Adu...
4    {'0': 'Adult 18+', '1': 'Adult 18+', '2': 'Tee...
Name: participant_age_group_map, dtype: object

# Find the unique type

In [7]:
def get_unique_name(df,column_name):
    s = set()
    for item in df[column_name]:
        for k,v in item.items():
            s.add(v)
    print(column_name,s)
get_unique_name(df,'participant_type_map') 
get_unique_name(df,'participant_age_group_map') 
get_unique_name(df,'participant_gender_map') 


participant_type_map {'Victim', 'Subject-Suspect'}
participant_age_group_map {'Adult 18+', 'Teen 12-17', 'Child 0-11'}
participant_gender_map {'Male', 'Male, female', 'Female'}


In [11]:
def get_suspect_gender(x, p_type):
    participant_type_map, participant_gender_map = x['participant_type_map'], x['participant_gender_map']
    suspect_id = {x for x in participant_type_map if participant_type_map[x] == p_type}
    suspect_gender = Counter([v for k, v in participant_gender_map.items() if k in suspect_id])
    
    return suspect_gender.most_common(1)[0][0] if suspect_gender else 'Male'

df["suspect_gender"] = df.apply(lambda x: get_suspect_gender(x, "Subject-Suspect"), axis=1)
df["victim_gender"] = df.apply(lambda x: get_suspect_gender(x, "Victim"), axis=1)

subdf = df[df['city_or_county'] == "Pittsburgh"].copy()
subdf = subdf[subdf['suspect_gender'] == "Male"]
subdf.head()

,Unnamed: 0,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url_fields_missing,congressional_district,...,participant_status,participant_type,state_house_district,state_senate_district,participant_type_map,participant_age_map,participant_age_group_map,participant_gender_map,suspect_gender,victim_gender
1587,1587,95997,2014-01-10,Pennsylvania,Pittsburgh,Craft Avenue and Boulevard of the Allies,0,0,False,14.0,...,NaN,0::Subject-Suspect,NaN,NaN,{'0': 'Subject-Suspect'},{},{},{},Male,Male
1591,1591,95999,2014-01-10,Pennsylvania,Pittsburgh,2000 block of Fifth Avenue,0,1,False,14.0,...,0::Injured,0::Victim,NaN,NaN,{'0': 'Victim'},{},{'0': 'Adult 18+'},{'0': 'Male'},Male,Male
2673,2673,98522,2014-01-17,Pennsylvania,Pittsburgh,Fifth Avenue and Kelly Street,0,1,False,14.0,...,0::Injured||1::Unharmed,0::Victim||1::Subject-Suspect,NaN,NaN,"{'0': 'Victim', '1': 'Subject-Suspect'}",{'0': '44'},{'0': 'Adult 18+'},{'0': 'Male'},Male,Male
3105,3105,98525,2014-01-20,Pennsylvania,Pittsburgh,Rapidan Way near Lowell Street,1,0,False,14.0,...,0::Killed||1::Unharmed,0::Victim||1::Subject-Suspect,NaN,NaN,"{'0': 'Victim', '1': 'Subject-Suspect'}","{'0': '37', '1': '29'}","{'0': 'Adult 18+', '1': 'Adult 18+'}","{'0': 'Male', '1': 'Male'}",Male,Male
3470,3470,99970,2014-01-23,Pennsylvania,Pittsburgh,Rosina Way,1,1,False,14.0,...,0::Killed||1::Injured,0::Victim||1::Victim,NaN,NaN,"{'0': 'Victim', '1': 'Victim'}","{'0': '79', '1': '79'}","{'0': 'Adult 18+', '1': 'Adult 18+'}","{'0': 'Female', '1': 'Male'}",Male,Female


In [9]:
df.to_csv('new_data.csv')